In [1]:
from colorthief import ColorThief
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import pandas as pd
import sys

sys.path.append("../")

plt.rcParams["figure.figsize"] = (20,10)

display(HTML("<style>.container { width:80% !important; }</style>"))

/tmp/ipykernel_18771/4170372904.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
file_path = (
    "../data/processed/sp_elected_est_fed_dep_tweets-analysis.parquet"
)
raw_df = pd.read_parquet(file_path)

print(raw_df.shape)
raw_df.head()

(125112, 29)


,id,date,content,user,in_reply_to_user,conversation_user,class_label,class_score,SG_UF,DS_CARGO,...,DS_COR_RACA,DS_OCUPACAO,DS_SIT_TOT_TURNO,ST_REELEICAO,TW_USER,TW_followersCount,TW_friendsCount,TW_statusesCount,TW_favouritesCount,user_is_candidate
0,1575873686019579905,2022-09-30 15:42:19+00:00,Na reta final nos encontramos novamente! @Hadd...,anaperugini,None,anaperugini,False,0.996524,SP,DEPUTADO ESTADUAL,...,BRANCA,SERVENTUÁRIO DE JUSTIÇA,ELEITO POR QP,N,anaperugini,5021,1038,10390,6916,True
1,1575873689039810565,2022-09-30 15:42:20+00:00,A caminhada será no centro de Hortolândia com ...,anaperugini,anaperugini,anaperugini,False,0.982824,SP,DEPUTADO ESTADUAL,...,BRANCA,SERVENTUÁRIO DE JUSTIÇA,ELEITO POR QP,N,anaperugini,5021,1038,10390,6916,True
2,1575943700139606020,2022-09-30 20:20:32+00:00,@anaperugini vou votar em você pela causa do t...,deaquilini,anaperugini,anaperugini,False,0.993578,SP,DEPUTADO ESTADUAL,...,BRANCA,SERVENTUÁRIO DE JUSTIÇA,ELEITO POR QP,N,anaperugini,5021,1038,10390,6916,False
3,1575876423885422592,2022-09-30 15:53:12+00:00,@anaperugini @Haddad_Fernando @geraldoalckmin ...,cliamariacardo1,anaperugini,anaperugini,False,0.959117,SP,DEPUTADO ESTADUAL,...,BRANCA,SERVENTUÁRIO DE JUSTIÇA,ELEITO POR QP,N,anaperugini,5021,1038,10390,6916,False
4,1575880426521960451,2022-09-30 16:09:06+00:00,@anaperugini @fmvasques53 @Haddad_Fernando @ge...,veracbrgc,anaperugini,anaperugini,False,0.988468,SP,DEPUTADO ESTADUAL,...,BRANCA,SERVENTUÁRIO DE JUSTIÇA,ELEITO POR QP,N,anaperugini,5021,1038,10390,6916,False


In [3]:
cols = ['user', 'conversation_user', ]
edges = raw_df[cols].groupby(cols, as_index=False).value_counts()

print(edges.shape)
edges['count'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

(68620, 3)


count    68620.000000
mean         1.823259
std          8.284465
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
90%          3.000000
95%          4.000000
99%         11.000000
max       1163.000000
Name: count, dtype: float64

In [4]:
# Filtrando somente conexões com uma candidata, para teste
# edges = edges[edges['conversation_user'] == 'andreawerner_']
# print(edges.shape)

# Removendo linhas de auto referenciação
edges = edges[edges['user'] != edges['conversation_user']]
print(edges.shape)

# Filtrando somente conexões com mais de 10 tweets
edges = edges[edges['count'] > 50]
print(edges.shape)

(68573, 3)
(21, 3)


In [261]:
# G = nx.from_pandas_edgelist(
#     edges, 
#     source='user',
#     target='conversation_user',
#     edge_attr='count',
#     create_using=nx.DiGraph()
# )
# print(G)

In [8]:
from pyvis.network import Network

nt = Network(
    height="600px", width="1200px", 
    directed=True, 
    notebook=True, 
    cdn_resources='remote'
)

users = edges['user']
c_users = edges['conversation_user']
counts = edges['count']
edge_data = zip(users, c_users, counts)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    nt.add_node(src, src, title=src)
    nt.add_node(dst, dst, title=dst)
    nt.add_edge(src, dst, value=w)

neighbor_map = nt.get_adj_list()

for node in nt.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

nt.show_buttons(filter_=['nodes'])
file_path = "../data/chart.html"
nt.show(file_path)

../data/chart.html
